# CNN con Keras Core

Utilizaremos keras core para clasificar imagenes de la base de datos MNIST

In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
!pip install -q keras-core
os.environ["KERAS_BACKEND"] = "torch"
# Nota: importar keras luego de elegir el backend torch, tf, jax
import keras_core as keras

Utlizaremos el dataset simplificado de MNIST de https://www.kaggle.com/datasets/oddrationale/mnist-in-csv, a continuacion lo descomprimimos en el directorio ./data/mnist/extracted/

In [ ]:
import zipfile
with zipfile.ZipFile("./data/mnist/mnist_train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("./data/mnist/extracted/")
with zipfile.ZipFile("./data/mnist/mnist_test.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("./data/mnist/extracted/")

Cargo los datasets de train y test y los convierto a la forma original de la imagen 28x28 pixeles

In [ ]:
train = pd.read_csv("./data/mnist/extracted/mnist_train.csv")


x_train = train.iloc[:,1:].values
y_train = train.iloc[:,0].values

# Hacer que cada pixel este entre 0 y 1
x_train = x_train.astype("float32") / 255

# Cambiar la forma de unidimensional de (784) a bidimensional (28, 28, 1)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)

In [ ]:
print(x_train[0].shape)
print(y_train[0])

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x_train[0], interpolation='nearest')
plt.show()

In [ ]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

Definimos la arquitectura de la CNN

In [ ]:
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

Compilamos el modelo

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

Entrenamos el modelo

In [ ]:
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=5,
    validation_split=0.15,
)

Ahora veamos como funciona la prediccion:

In [ ]:
test = pd.read_csv("./data/mnist/extracted/mnist_test.csv")

x_test = test.iloc[:,1:].values
y_test = test.iloc[:,0].values

# Hacer que cada pixel este entre 0 y 1
x_test = x_test.astype("float32") / 255

# Cambiar la forma de unidimensional de (784) a bidimensional (28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x_test[0], interpolation='nearest')
plt.show()

In [ ]:
predictions = model.predict(x_test[0:1,:])

In [ ]:
predictions

In [ ]:
predictions = predictions.argmax(axis=-1).flatten()

In [ ]:
predictions